In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession\
    .builder\
    .appName("twitter_transformation")\
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble


23/05/26 22:25:51 WARN Utils: Your hostname, BIRIBA resolves to a loopback address: 127.0.1.1; using 172.29.6.247 instead (on interface eth0)
23/05/26 22:25:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/26 22:25:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = spark.read.json("../datalake/twitter_datascience")

In [5]:
df.show()

+--------------------+--------------------+------------+
|                data|            includes|extract_date|
+--------------------+--------------------+------------+
|[{93, 29, 2023-05...|{[{2023-05-25T20:...|  2023-05-25|
|[{49, 35, 2023-05...|{[{2023-05-26T00:...|  2023-05-26|
+--------------------+--------------------+------------+



In [7]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- c

In [8]:
from pyspark.sql import functions as f

In [9]:
df.select(f.explode("data")).printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- author_id: string (nullable = true)
 |    |-- conversation_id: string (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- id: string (nullable = true)
 |    |-- in_reply_to_user_id: string (nullable = true)
 |    |-- lang: string (nullable = true)
 |    |-- public_metrics: struct (nullable = true)
 |    |    |-- like_count: long (nullable = true)
 |    |    |-- quote_count: long (nullable = true)
 |    |    |-- reply_count: long (nullable = true)
 |    |    |-- retweet_count: long (nullable = true)
 |    |-- text: string (nullable = true)



In [10]:
df.select(f.explode("data")).show()

+--------------------+
|                 col|
+--------------------+
|{93, 29, 2023-05-...|
|{25, 7, 2023-05-2...|
|{69, 85, 2023-05-...|
|{35, 91, 2023-05-...|
|{55, 28, 2023-05-...|
|{66, 5, 2023-05-2...|
|{0, 45, 2023-05-2...|
|{74, 49, 2023-05-...|
|{0, 18, 2023-05-2...|
|{86, 60, 2023-05-...|
|{49, 35, 2023-05-...|
|{54, 53, 2023-05-...|
|{100, 36, 2023-05...|
|{57, 24, 2023-05-...|
|{76, 65, 2023-05-...|
|{81, 7, 2023-05-2...|
|{97, 47, 2023-05-...|
|{40, 77, 2023-05-...|
|{20, 65, 2023-05-...|
|{27, 89, 2023-05-...|
+--------------------+



In [12]:
tweet_df = df.select(f.explode("data").alias("tweets"))\
  .select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text")

In [13]:
tweet_df.show(5)

+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|author_id|conversation_id|          created_at| id|like_count|quote_count|reply_count|retweet_count|                text|
+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|       93|             29|2023-05-25T22:09:...| 48|        11|         98|         89|           98|Este é um tweet f...|
|       25|              7|2023-05-25T14:00:...| 86|        41|         63|          2|           41|Tweet fictício ge...|
|       69|             85|2023-05-25T05:17:...| 78|        63|         48|         74|           92|Um terceiro tweet...|
|       35|             91|2023-05-25T00:17:...| 20|        96|         27|         23|           85|Outro tweet fictí...|
|       55|             28|2023-05-25T02:38:...| 10|        81|         81|         41|           39|Um terceiro tweet...|
+---------+-----

In [14]:
df.select(f.explode("includes.users")).printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- username: string (nullable = true)



In [15]:
user_df = df.select(f.explode("includes.users").alias("users")).select("users.*")

In [16]:
user_df.show(5)

+--------------------+---+------+--------+
|          created_at| id|  name|username|
+--------------------+---+------+--------+
|2023-05-25T20:34:...| 43|User 1|   user1|
|2023-05-25T11:05:...| 55|User 2|   user2|
|2023-05-25T22:21:...| 99|User 3|   user3|
|2023-05-25T13:22:...| 61|User 4|   user4|
|2023-05-25T08:27:...|  5|User 5|   user5|
+--------------------+---+------+--------+
only showing top 5 rows



In [17]:
tweet_df.coalesce(1).write.mode("overwrite").json('output/tweet')

In [18]:
tweet_df.coalesce(1).write.mode("overwrite").json('output/tweet')
user_df.coalesce(1).write.mode("overwrite").json('output/user')